In [1]:
using TSSOS
using DynamicPolynomials
using CSV
using DataFrames
using Statistics

In [2]:
#dis = CSV.read("distanceSet.csv", header=1);
taxi=[1.38,0.77,2.52,1.15,1.2,3.53,1.0,2.2,0.95,1.42,1.7,2.77,0.57, 0.81,21.47, 1.13,1.19, 0.83, 0.93, 3.3,0.71, 1.04, 11.2, 0.72, 2.63, 0.82, 10.32, 3.51,7.25, 0.48, 2.5, 1.5, 0.4, 1.86, 2.34, 1.42,2.6,1.4, 1.02, 5.96, 0.84, 1.2, 9.3, 1.66,2.0, 2.55, 2.04, 17.87,0.6, 0.57];

In [3]:
# new formulation 3 10/02/21
# calculate run time

function F3_tssos_21(icount,jcount,taxi,lam1,lam2,lam3)
    #icount=5 # the number of customers
    #jcount=5 # the number of drivers
    #female=[2,4]
    #male=[1,3,5]
    female=collect(2:2:jcount)
    male=collect(1:2:jcount)
    
    mu1=10
    mu2=10
    mu3=0

    #lam1=1
    #lam2=1
    #lam3=0
    
    dis=rand(icount,jcount)
    
    # set nc operators
    @polyvar u[1:jcount] M[1:icount*jcount]; #z[1:jcount]; 
    var=vcat(u,M);

    #l2=[z[j]-1+(u[j]+0.01)/1.01 for j in 1:jcount];
    #l3=[z[j]+1-(u[j]+0.01)/1.01 for j in 1:jcount];
    #l4=[b1-(1/length(female))*sum(z[j] for j in female),b1-(1/length(male))*sum(z[j] for j in male)];
    #l5=[b2-sum(M[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount) for i in 1:icount];
    l6=[1-sum(M[(i-1)*(jcount)+j] for i in 1:icount) for j in 1:jcount];
    #l7=[M[(i-1)*(jcount)+j]*(M[(i-1)*(jcount)+j]-1) for i in 1:icount for j in 1:jcount]; #equality #numeq=25
    l8=[u[j] for j in 1:jcount];
    l9=[M[(i-1)*(jcount)+j] for i in 1:icount for j in 1:jcount];

    obj=lam1*sum((1-(u[j]+0.01)/1.01)^2 for j in 1:jcount)+lam2*(sum((u[j]+0.01)/1.01 for j in female)-sum((u[j]+0.01)/1.01 for j in male))^2+lam3*sum(M[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount for i in 1:icount)+sum(mu1*(sum(M[(i-1)*(jcount)+j]*(taxi[i]-dis[i,j]) for i in 1:icount)-u[j])^2  for j in 1:jcount)+sum(mu2*(sum(M[(i-1)*(jcount)+j] for j in 1:jcount)-1)^2  for i in 1:icount)+sum(M[(i-1)*(jcount)+j]*(M[(i-1)*(jcount)+j]-1) for i in 1:icount for j in 1:jcount)

    # pop
    pop=vcat(obj,l6,l8,l9); #,l7

    d=1; # the relaxation order
    runtime=@elapsed cs_tssos_first(pop,var,d,numeq=icount*jcount,TS="MD",solution=false);
    #opt,sol,data=cs_tssos_higher!(data,TS="MD")
    
    #u_sol=sol[1:jcount]
    #M_sol=sol[jcount+1:end]
    #intra=sum((1-(u_sol[j]+0.01)/1.01)^2 for j in 1:jcount)
    #inter=(sum((u_sol[j]+0.01)/1.01 for j in female)-sum((u_sol[j]+0.01)/1.01 for j in male))^2
    #care=sum(M_sol[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount for i in 1:icount)
    
    return runtime
    #[intra,inter,care]

end

F3_tssos_21 (generic function with 1 method)

In [9]:
df_run_21

,size,runInter0,runInter3
,Int64,Float64,Float64
1,5,43.6258,0.36359
2,5,0.404184,0.319799
3,5,0.257352,0.251441
4,5,0.564133,0.365502
5,5,0.226636,0.416875
6,5,0.253198,0.279661
7,5,0.265516,0.313144
8,5,0.308409,0.421281
9,5,0.422874,0.335671


In [6]:
# for runtime 
#push!(df, push!(obj,lam1,lam2,1-lam1-lam2))
repeat=10
size_range=5:1:10
df_run_21 = DataFrame(size=Int64[],runInter0=Float64[],runInter3=Float64[])
for t in size_range
    for r in 1:repeat
        runInter0=F3_tssos_21(t,t,taxi[1:t],1,0,0);
        runInter3=F3_tssos_21(t,t,taxi[1:t],0.5,0.5,0);
        push!(df_run_21,(t,runInter0,runInter3))
    end
end

------------------------------------------------------
The clique sizes of varibles:
[16, 14, 9, 6]
[1, 2, 5, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 296             
  Cones                  : 0               
  Scalar variables       : 36              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve term

3   5.5e+00  9.0e-02  1.5e-01  -5.83e-01  -7.875175946e+01  -8.123960143e+01  9.0e-02  0.09  
4   4.7e+00  7.6e-02  1.3e-01  1.04e+00   5.754716790e+00   3.230943473e+00   7.6e-02  0.11  
5   1.5e+00  2.4e-02  1.5e-02  1.10e+00   5.004767794e+01   4.976126914e+01   2.4e-02  0.13  
6   1.5e-01  2.4e-03  3.6e-04  1.93e+00   5.346261964e+01   5.344602143e+01   2.4e-03  0.14  
7   4.6e-03  7.5e-05  1.9e-06  1.05e+00   5.442975512e+01   5.442926550e+01   7.5e-05  0.16  
8   1.4e-04  2.3e-06  1.0e-08  1.00e+00   5.446267964e+01   5.446266458e+01   2.3e-06  0.16  
9   2.7e-06  4.4e-08  2.8e-11  1.00e+00   5.446386162e+01   5.446386134e+01   4.4e-08  0.19  
10  1.6e-08  2.6e-10  1.2e-14  1.00e+00   5.446388700e+01   5.446388700e+01   2.6e-10  0.20  
Optimizer terminated. Time: 0.23    

optimum = 54.463886998538655
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  N

3   4.9e+00  8.7e-02  1.5e-01  -5.49e-01  -7.727825055e+01  -7.968120133e+01  8.7e-02  0.09  
4   4.1e+00  7.2e-02  1.2e-01  9.79e-01   9.747603003e+00   7.455591632e+00   7.2e-02  0.11  
5   1.3e+00  2.3e-02  1.3e-02  1.07e+00   4.894112003e+01   4.871823307e+01   2.3e-02  0.11  
6   2.6e-01  4.5e-03  9.2e-04  2.07e+00   5.081569502e+01   5.078434679e+01   4.5e-03  0.13  
7   5.4e-03  9.6e-05  2.6e-06  1.13e+00   5.230091416e+01   5.230037402e+01   9.6e-05  0.14  
8   1.8e-04  3.2e-06  1.6e-08  1.00e+00   5.233627625e+01   5.233625855e+01   3.2e-06  0.16  
9   4.8e-06  8.5e-08  6.8e-11  1.00e+00   5.233747405e+01   5.233747358e+01   8.5e-08  0.19  
10  1.1e-07  2.0e-09  2.4e-13  1.00e+00   5.233750786e+01   5.233750785e+01   2.0e-09  0.20  
Optimizer terminated. Time: 0.22    

optimum = 52.33750785871106
------------------------------------------------------
The clique sizes of varibles:
[16, 14, 9, 6]
[1, 2, 5, 5]
------------------------------------------------------
Problem
  Name

Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 296             
  Cones                  : 0               
  Scalar variables       : 36              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 296
Optimizer  - Cones                  : 1
Optimizer  - Scalar variables       : 37                conic                  : 27              
Optimizer  - Semi-definite variables: 52                scalarized             : 1690            
Factor     - setup time             : 0.02              dense det. time        : 0.00            
Factor     - ML order time          : 0.00              GP order time          : 0.00            
Factor     - nonzeros before factor :

3   5.6e+00  9.2e-02  1.6e-01  -5.76e-01  -9.277022926e+01  -9.544086820e+01  9.2e-02  0.09  
4   4.7e+00  7.7e-02  1.3e-01  1.24e+00   2.706082673e+00   2.506673137e-01   7.7e-02  0.11  
5   1.4e+00  2.3e-02  1.4e-02  1.09e+00   4.961004111e+01   4.935257041e+01   2.3e-02  0.13  
6   1.5e-01  2.5e-03  4.0e-04  1.95e+00   5.333248877e+01   5.331194617e+01   2.5e-03  0.14  
7   4.1e-03  6.8e-05  1.8e-06  1.05e+00   5.443070632e+01   5.443017382e+01   6.8e-05  0.16  
8   1.3e-04  2.1e-06  9.5e-09  1.00e+00   5.446275591e+01   5.446273954e+01   2.1e-06  0.17  
9   2.2e-06  3.6e-08  2.2e-11  1.00e+00   5.446386600e+01   5.446386571e+01   3.6e-08  0.19  
10  1.3e-08  2.1e-10  9.3e-15  1.00e+00   5.446388703e+01   5.446388702e+01   2.1e-10  0.20  
Optimizer terminated. Time: 0.22    

optimum = 54.46388702656525
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Na

...

1   6.7e+01  4.4e-01  6.2e-01  -9.23e-01  -5.723712947e+01  -5.833740584e+01  4.4e-01  3.13  
2   2.8e+01  1.8e-01  3.6e-01  -8.07e-01  -3.882356010e+02  -3.911262003e+02  1.8e-01  4.72  
3   1.0e+01  6.7e-02  1.2e-01  -4.81e-01  -3.877565201e+02  -3.903358650e+02  6.7e-02  6.33  
4   9.1e+00  5.9e-02  9.3e-02  1.41e+00   -1.376587602e+02  -1.397513613e+02  5.9e-02  7.86  
5   3.9e+00  2.5e-02  1.2e-02  1.62e+00   8.129652635e+01   8.115241199e+01   2.5e-02  9.47  
6   5.2e-01  3.4e-03  5.9e-04  2.64e+00   1.004162475e+02   1.003903444e+02   3.4e-03  11.23 
7   3.6e-03  2.3e-05  9.0e-08  1.17e+00   1.046630085e+02   1.046630221e+02   2.3e-05  13.05 
8   1.8e-06  1.1e-08  9.6e-13  1.00e+00   1.046724553e+02   1.046724553e+02   1.1e-08  14.84 
9   1.4e-10  9.4e-12  6.0e-19  1.00e+00   1.046724605e+02   1.046724605e+02   7.9e-13  16.64 
Optimizer terminated. Time: 16.67   

optimum = 104.67246051726553
------------------------------------------------------
The clique sizes of varibles:
[6

Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3666            
  Cones                  : 0               
  Scalar variables       : 121             
  Matrix variables       : 202             
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 3666
Optimizer  - Cones                  : 1
Optimizer  - Scalar variables       : 122               conic                  : 102             
Optimizer  - Semi-definite variables: 202               scalarized             : 27425           
Factor     - setup time             : 0.72              dense det. time        : 0.00            
Factor     - ML order time          : 0.19              GP order time          : 0.00            
Factor     - nonzeros before factor 

7   3.2e-07  2.2e-09  1.3e-13  1.00e+00   1.089277714e+02   1.089277714e+02   2.2e-09  8.31  
Optimizer terminated. Time: 8.34    

optimum = 108.92777137229399
------------------------------------------------------
The clique sizes of varibles:
[72, 60, 43, 36, 28, 20]
[1, 2, 1, 4, 1, 10]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 4281            
  Cones                  : 0               
  Scalar variables       : 121             
  Matrix variables       : 202             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00 

1   6.5e+01  4.4e-01  6.3e-01  -9.27e-01  -5.441891771e+01  -5.550904651e+01  4.4e-01  3.09  
2   2.9e+01  1.9e-01  3.7e-01  -8.11e-01  -3.746347547e+02  -3.774302507e+02  1.9e-01  4.69  
3   8.8e+00  5.9e-02  8.7e-02  -5.07e-01  -2.466563775e+02  -2.484600056e+02  5.9e-02  6.41  
4   6.9e+00  4.7e-02  5.3e-02  1.67e+00   1.841572795e+01   1.734677016e+01   4.7e-02  7.97  
5   2.1e+00  1.4e-02  3.6e-03  2.16e+00   1.024790226e+02   1.024425322e+02   1.4e-02  9.66  
6   9.2e-02  6.2e-04  4.3e-05  1.86e+00   1.041421841e+02   1.041382053e+02   6.2e-04  11.31 
7   8.1e-05  5.4e-07  1.1e-09  1.04e+00   1.046719510e+02   1.046719478e+02   5.4e-07  13.11 
8   3.6e-09  2.4e-11  3.2e-16  1.00e+00   1.046724605e+02   1.046724605e+02   2.4e-11  14.94 
Optimizer terminated. Time: 14.97   

optimum = 104.6724604947774
------------------------------------------------------
The clique sizes of varibles:
[64, 56, 34, 19, 11]
[1, 2, 5, 10, 10]
------------------------------------------------------
Pro

Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3666            
  Cones                  : 0               
  Scalar variables       : 121             
  Matrix variables       : 202             
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 3666
Optimizer  - Cones                  : 1
Optimizer  - Scalar variables       : 122               conic                  : 102             
Optimizer  - Semi-definite variables: 202               scalarized             : 27425           
Factor     - setup time             : 0.72              dense det. time        : 0.00            
Factor     - ML order time          : 0.19              GP order time          : 0.00            
Factor     - nonzeros before factor 

In [8]:
CSV.write("SDPtime_F2_21.csv", df_run_21,header=true)

"SDPtime_F2_21.csv"

In [3]:
# new formulation 3 19/11
# calculate run time

function F3_tssos(icount,jcount,taxi,lam1,lam2,lam3)
    #icount=5 # the number of customers
    #jcount=5 # the number of drivers
    #female=[2,4]
    #male=[1,3,5]
    female=collect(2:2:jcount)
    male=collect(1:2:jcount)
    
    mu1=10
    mu2=10
    mu3=0

    #lam1=1
    #lam2=1
    #lam3=0
    
    dis=rand(icount,jcount)
    
    # set nc operators
    @polyvar u[1:jcount] M[1:icount*jcount]; #z[1:jcount]; 
    var=vcat(u,M);

    #l2=[z[j]-1+(u[j]+0.01)/1.01 for j in 1:jcount];
    #l3=[z[j]+1-(u[j]+0.01)/1.01 for j in 1:jcount];
    #l4=[b1-(1/length(female))*sum(z[j] for j in female),b1-(1/length(male))*sum(z[j] for j in male)];
    #l5=[b2-sum(M[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount) for i in 1:icount];
    l6=[1-sum(M[(i-1)*(jcount)+j] for i in 1:icount) for j in 1:jcount];
    l7=[M[(i-1)*(jcount)+j]*(M[(i-1)*(jcount)+j]-1) for i in 1:icount for j in 1:jcount]; #equality #numeq=25

    obj=lam1*sum((1-(u[j]+0.01)/1.01)^2 for j in 1:jcount)+lam2*(sum((u[j]+0.01)/1.01 for j in female)-sum((u[j]+0.01)/1.01 for j in male))^2+lam3*sum(M[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount for i in 1:icount)+sum(mu1*(sum(M[(i-1)*(jcount)+j]*(taxi[i]-dis[i,j]) for i in 1:icount)-u[j])^2  for j in 1:jcount)+sum(mu2*(sum(M[(i-1)*(jcount)+j] for j in 1:jcount)-1)^2  for i in 1:icount)

    # pop
    pop=vcat(obj,l6,l7);

    d=1; # the relaxation order
    runtime=@elapsed cs_tssos_first(pop,var,d,numeq=icount*jcount,TS="MD",solution=false);
    #opt,sol,data=cs_tssos_higher!(data,TS="MD")
    
    #u_sol=sol[1:jcount]
    #M_sol=sol[jcount+1:end]
    #intra=sum((1-(u_sol[j]+0.01)/1.01)^2 for j in 1:jcount)
    #inter=(sum((u_sol[j]+0.01)/1.01 for j in female)-sum((u_sol[j]+0.01)/1.01 for j in male))^2
    #care=sum(M_sol[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount for i in 1:icount)
    
    return runtime
    #[intra,inter,care]

end

F3_tssos (generic function with 1 method)

In [6]:
# for runtime 
#push!(df, push!(obj,lam1,lam2,1-lam1-lam2))
repeat=10
size_range=5:1:10
df_run = DataFrame(size=Int64[],runInter0=Float64[],runInter3=Float64[])
for t in size_range
    for r in 1:repeat
        runInter0=F3_tssos(t,t,taxi[1:t],1,0,0);
        runInter3=F3_tssos(t,t,taxi[1:t],0.5,0.5,0);
        push!(df_run,(t,runInter0,runInter3))
    end
end

------------------------------------------------------
The clique sizes of varibles:
[16, 14, 9, 6]
[1, 2, 5, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 296             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve term

1   3.6e+01  6.8e-01  8.0e-01  -8.81e-01  -2.826040779e+00  -3.240319082e+00  6.8e-01  0.03  
2   2.8e+01  5.3e-01  7.0e-01  -9.24e-01  -4.447400561e+01  -4.524174424e+01  5.3e-01  0.06  
3   8.7e+00  1.6e-01  3.4e-01  -8.63e-01  -2.294046467e+02  -2.329375155e+02  1.6e-01  0.09  
4   4.2e+00  7.8e-02  7.8e-02  -1.04e-02  -1.495139700e+02  -1.501604652e+02  7.8e-02  0.11  
5   1.1e+00  2.0e-02  4.2e-03  1.72e+00   -1.981400175e+01  -1.980330972e+01  2.0e-02  0.13  
6   2.3e-01  4.3e-03  4.4e-04  1.32e+00   -3.524944956e+00  -3.525979456e+00  4.3e-03  0.13  
7   4.6e-02  8.6e-04  3.9e-05  1.05e+00   -7.450778943e-01  -7.453254213e-01  8.6e-04  0.17  
8   1.0e-02  1.9e-04  4.7e-06  1.01e+00   -1.820197117e-01  -1.822449366e-01  1.9e-04  0.20  
9   3.1e-03  5.8e-05  8.4e-07  1.00e+00   -5.725673027e-02  -5.734012230e-02  5.8e-05  0.25  
10  7.2e-04  1.3e-05  9.3e-08  1.00e+00   -1.342401395e-02  -1.344377976e-02  1.3e-05  0.27  
11  1.9e-04  3.5e-06  1.3e-08  9.99e-01   -3.639431857e-03  

9   5.9e-03  1.1e-04  2.6e-06  1.00e+00   -1.040170214e-01  -1.043195996e-01  1.1e-04  0.28  
10  1.5e-03  2.8e-05  3.3e-07  1.00e+00   -2.488198425e-02  -2.495150576e-02  2.8e-05  0.31  
11  3.3e-04  6.0e-06  3.2e-08  1.00e+00   -2.315160956e-03  -2.329170383e-03  6.0e-06  0.36  
12  4.4e-05  8.2e-07  1.6e-09  1.00e+00   3.335082932e-03   3.333211242e-03   8.2e-07  0.38  
13  4.5e-06  8.2e-08  5.0e-11  1.00e+00   4.205846145e-03   4.205657766e-03   8.2e-08  0.39  
14  6.2e-07  1.2e-08  2.6e-12  1.00e+00   4.289496154e-03   4.289470129e-03   1.1e-08  0.41  
15  9.8e-08  6.3e-09  1.7e-13  1.00e+00   4.300591341e-03   4.300587139e-03   1.8e-09  0.41  
Optimizer terminated. Time: 0.42    

optimum = 0.004300591340704187
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type 

------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max        

2   2.6e+01  5.1e-01  6.8e-01  -9.04e-01  -4.786529804e+01  -4.865231971e+01  5.1e-01  0.14  
3   8.4e+00  1.7e-01  3.1e-01  -8.02e-01  -1.879481581e+02  -1.906487810e+02  1.7e-01  0.16  
4   3.5e+00  7.0e-02  5.8e-02  4.67e-01   -7.539454755e+01  -7.584905716e+01  7.0e-02  0.20  
5   8.0e-01  1.6e-02  4.0e-03  1.51e+00   -1.438967400e+01  -1.441857732e+01  1.6e-02  0.23  
6   2.0e-01  4.0e-03  4.9e-04  1.29e+00   -2.524289479e+00  -2.531254745e+00  4.0e-03  0.27  
7   3.3e-02  6.6e-04  3.1e-05  1.09e+00   -2.418463425e-01  -2.428154785e-01  6.6e-04  0.28  
8   7.8e-03  1.5e-04  3.4e-06  1.01e+00   4.426661285e-02   4.406086186e-02   1.5e-04  0.31  
9   2.3e-03  4.5e-05  5.5e-07  9.98e-01   9.627409105e-02   9.621238719e-02   4.5e-05  0.33  
10  7.1e-04  1.4e-05  9.5e-08  9.97e-01   1.104886607e-01   1.104693179e-01   1.4e-05  0.34  
11  1.8e-04  3.6e-06  1.2e-08  9.97e-01   1.150162668e-01   1.150114160e-01   3.6e-06  0.36  
12  3.4e-05  6.6e-07  9.6e-10  9.98e-01   1.160433832e-01   

...

2   1.1e+02  7.5e-01  8.6e-01  -9.57e-01  -6.477155707e+01  -6.508541499e+01  7.5e-01  3.59  
3   9.2e+01  6.2e-01  7.7e-01  -9.53e-01  -1.498281249e+02  -1.503898544e+02  6.2e-01  4.84  
4   3.7e+01  2.5e-01  4.6e-01  -9.40e-01  -9.366284809e+02  -9.391842239e+02  2.5e-01  6.09  
5   1.8e+01  1.2e-01  2.3e-01  -6.29e-01  -1.355388444e+03  -1.358227888e+03  1.2e-01  7.34  
6   1.3e+01  8.6e-02  6.0e-02  1.21e+00   -5.362291720e+02  -5.364185836e+02  8.6e-02  8.59  
7   2.2e+00  1.5e-02  1.8e-03  1.76e+00   -5.786025100e+01  -5.785311767e+01  1.5e-02  9.98  
8   3.4e-01  2.3e-03  1.6e-04  1.36e+00   -8.306253979e+00  -8.308216132e+00  2.3e-03  11.31 
9   3.9e-02  2.6e-04  5.5e-06  1.06e+00   -9.164064841e-01  -9.165142923e-01  2.6e-04  12.69 
10  1.6e-02  1.0e-04  1.4e-06  1.01e+00   -3.789315788e-01  -3.789663008e-01  1.0e-04  13.97 
11  2.4e-03  1.6e-05  8.0e-08  1.00e+00   -6.058846825e-02  -6.059258947e-02  1.6e-05  15.38 
12  1.5e-03  1.0e-05  3.9e-08  1.00e+00   -3.735672196e-02  

4   3.7e+01  2.4e-01  4.6e-01  -9.40e-01  -9.695463945e+02  -9.721894261e+02  2.4e-01  6.03  
5   1.8e+01  1.2e-01  2.0e-01  -6.04e-01  -1.231000021e+03  -1.233418328e+03  1.2e-01  7.30  
6   1.2e+01  7.9e-02  5.3e-02  1.47e+00   -4.716086993e+02  -4.718027401e+02  7.9e-02  8.53  
7   3.3e+00  2.1e-02  4.1e-03  1.89e+00   -8.491802415e+01  -8.492013050e+01  2.1e-02  9.84  
8   4.4e-01  2.9e-03  2.5e-04  1.46e+00   -1.062909167e+01  -1.063291787e+01  2.9e-03  11.30 
9   6.8e-02  4.4e-04  1.2e-05  1.07e+00   -1.617235700e+00  -1.617436630e+00  4.4e-04  12.67 
10  3.7e-02  2.4e-04  4.9e-06  1.01e+00   -8.883771661e-01  -8.884825339e-01  2.4e-04  13.91 
11  3.9e-03  2.5e-05  1.6e-07  1.01e+00   -9.269601534e-02  -9.270220250e-02  2.5e-05  15.31 
12  2.9e-03  1.9e-05  9.9e-08  1.00e+00   -6.861356422e-02  -6.861829695e-02  1.9e-05  16.56 
13  3.9e-04  2.5e-06  4.9e-09  1.00e+00   -9.421021695e-03  -9.421665166e-03  2.5e-06  17.94 
14  1.5e-04  9.5e-07  1.2e-09  1.00e+00   -3.555712501e-03  

7   2.2e+00  1.5e-02  2.0e-03  1.73e+00   -6.374807758e+01  -6.374079140e+01  1.5e-02  9.88  
8   3.4e-01  2.3e-03  1.7e-04  1.37e+00   -9.291136205e+00  -9.293537114e+00  2.3e-03  11.23 
9   2.1e-01  1.4e-03  7.9e-05  1.06e+00   -5.494419579e+00  -5.495745243e+00  1.4e-03  12.42 
10  1.6e-02  1.1e-04  1.3e-06  1.04e+00   -3.665625669e-01  -3.665824712e-01  1.1e-04  13.86 
11  1.1e-02  7.8e-05  8.4e-07  1.00e+00   -2.802312362e-01  -2.802473849e-01  7.8e-05  15.30 
12  5.1e-03  3.5e-05  2.5e-07  1.00e+00   -1.315350670e-01  -1.315422158e-01  3.5e-05  16.56 
13  3.5e-04  2.4e-06  4.2e-09  1.00e+00   -9.472222305e-03  -9.472206162e-03  2.4e-06  18.02 
14  1.4e-04  9.8e-07  1.2e-09  1.00e+00   -3.949096842e-03  -3.949204122e-03  9.8e-07  19.33 
15  1.1e-05  7.5e-08  2.6e-11  1.00e+00   -3.009779747e-04  -3.009979324e-04  7.5e-08  20.73 
16  1.0e-06  7.5e-09  7.4e-13  1.00e+00   -2.787578468e-05  -2.787829244e-05  6.9e-09  22.22 
Optimizer terminated. Time: 22.28   

optimum = -2.787578467

8   4.8e-01  3.5e-03  3.5e-04  1.51e+00   -1.310667189e+01  -1.311228869e+01  3.5e-03  11.23 
9   6.0e-02  4.3e-04  1.2e-05  1.08e+00   -1.623351076e+00  -1.623575187e+00  4.3e-04  12.67 
10  2.5e-02  1.8e-04  3.3e-06  1.01e+00   -6.878204186e-01  -6.879060337e-01  1.8e-04  13.95 
11  1.1e-02  7.8e-05  9.1e-07  1.00e+00   -2.961925286e-01  -2.962253385e-01  7.8e-05  15.23 
12  1.0e-03  7.3e-06  2.4e-08  1.00e+00   -2.840758748e-02  -2.840876500e-02  7.3e-06  16.70 
13  8.7e-04  6.3e-06  1.9e-08  9.99e-01   -2.446787504e-02  -2.446895076e-02  6.3e-06  17.95 
14  4.1e-04  3.0e-06  6.3e-09  1.00e+00   -1.165172666e-02  -1.165240352e-02  3.0e-06  19.23 
15  5.5e-05  4.0e-07  3.2e-10  1.00e+00   -1.538294325e-03  -1.538429154e-03  4.0e-07  20.59 
16  2.5e-05  1.8e-07  1.0e-10  1.00e+00   -7.145234261e-04  -7.145869863e-04  1.8e-07  21.86 
17  2.1e-06  1.8e-08  2.5e-12  1.00e+00   -6.047213592e-05  -6.047723380e-05  1.6e-08  23.25 
18  1.6e-06  1.6e-08  1.6e-12  1.00e+00   -4.542957376e-05  

4   3.7e+01  2.5e-01  4.7e-01  -9.46e-01  -9.538347725e+02  -9.564317223e+02  2.5e-01  6.01  
5   1.7e+01  1.2e-01  2.1e-01  -6.73e-01  -1.251361077e+03  -1.253934578e+03  1.2e-01  7.31  
6   7.5e+00  5.0e-02  3.2e-02  1.25e+00   -3.345687538e+02  -3.348252871e+02  5.0e-02  8.59  
7   3.7e+00  2.5e-02  8.8e-03  2.24e+00   -9.259172123e+01  -9.267275740e+01  2.5e-02  9.81  
8   4.2e-01  2.8e-03  2.6e-04  1.45e+00   -1.172822999e+01  -1.173275816e+01  2.8e-03  11.25 
9   5.9e-02  3.9e-04  1.1e-05  1.07e+00   -1.525613676e+00  -1.525884844e+00  3.9e-04  12.59 
10  2.4e-02  1.6e-04  2.8e-06  1.01e+00   -6.367823099e-01  -6.368773568e-01  1.6e-04  13.88 
11  1.4e-02  9.1e-05  1.2e-06  1.00e+00   -3.635552130e-01  -3.636056127e-01  9.1e-05  15.14 
12  7.2e-03  4.8e-05  4.6e-07  1.00e+00   -1.942304549e-01  -1.942563132e-01  4.8e-05  16.36 
13  1.5e-03  1.0e-05  4.4e-08  1.00e+00   -4.215656891e-02  -4.216142797e-02  1.0e-05  17.45 
14  1.2e-04  7.7e-07  8.8e-10  1.00e+00   -3.178181470e-03  

In [7]:
df_run

,size,runInter0,runInter3
,Int64,Float64,Float64
1,5,0.413331,0.483084
2,5,0.472007,0.798289
3,5,0.556072,0.589421
4,5,0.590947,0.651348
5,5,0.443562,0.684161
6,5,0.545577,0.704997
7,5,0.447377,0.572073
8,5,0.531684,0.692894
9,5,0.510699,0.614333


In [8]:
CSV.write("SDPtime_F2.csv", df_run,header=true)

"SDPtime_F2.csv"